# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report
import cv2
import os
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

print("GPU Available:", tf.config.list_physical_devices('GPU'))

print(get_built_info())

In [4]:
import os
import cv2
import numpy as np

# Dataset path and image config
base_path = r"C:\Users\diogo\OneDrive\Documents\MEIA\AAUT2IA\data"
img_size = 128

X, y = [], []

for label in os.listdir(base_path):
    folder = os.path.join(base_path, label)
    if not os.path.isdir(folder): continue

    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, (img_size, img_size))

        # Basic contrast enhancement (Y-channel histogram equalization)
        img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
        img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
        img_eq = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

        img_norm = img_eq / 255.0  # Normalize pixel values
        X.append(img_norm)
        y.append(label)

X = np.array(X)
y = np.array(y)

print(f"✅ Loaded {X.shape[0]} images. Image shape: {X.shape[1:]}")

✅ Loaded 86421 images. Image shape: (128, 128, 3)


In [5]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Label encoding + one-hot
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_onehot = to_categorical(y_encoded)

# Show mapping
label_map = dict(zip(le.classes_, le.transform(le.classes_)))
print("📌 Label Mapping:", label_map)

📌 Label Mapping: {np.str_('Mild Dementia'): np.int64(0), np.str_('Moderate Dementia'): np.int64(1), np.str_('Non Demented'): np.int64(2), np.str_('Very mild Dementia'): np.int64(3)}


# Displaying clearly representative images from each class
fig, axes = plt.subplots(len(classes), 4, figsize=(15, 12))

for i, cls in enumerate(classes):
    class_path = os.path.join(base_path, cls)
    sample_images = os.listdir(class_path)[:4]  # Clearly select first 4 images for consistency

    for j, img_file in enumerate(sample_images):
        img = Image.open(os.path.join(class_path, img_file))
        axes[i, j].imshow(img, cmap='gray')
        axes[i, j].axis('off')
        axes[i, j].set_title(f"{cls} ({img.size[0]}x{img.size[1]})")

plt.tight_layout()
plt.show()

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y_onehot, test_size=0.3, stratify=y_encoded, random_state=42
)

print(f"🔁 Train: {X_train.shape}, Validation: {X_val.shape}")


🔁 Train: (60494, 128, 128, 3), Validation: (25927, 128, 128, 3)


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

augmentor = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen_custom = augmentor.flow(X_train, y_train, batch_size=64, shuffle=True)
steps_custom = train_gen_custom.n // train_gen_custom.batch_size

train_gen_vgg = augmentor.flow(X_train, y_train, batch_size=64, shuffle=True)
steps_vgg = train_gen_vgg.n // train_gen_vgg.batch_size

train_gen_resnet = augmentor.flow(X_train, y_train, batch_size=64, shuffle=True)
steps_resnet = train_gen_resnet.n // train_gen_resnet.batch_size

In [8]:
def build_custom_cnn(input_shape=(128, 128, 3), num_classes=4):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2, 2),
        BatchNormalization(),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        BatchNormalization(),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        BatchNormalization(),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

optimized_cnn = build_custom_cnn()
optimized_cnn.summary()

C:\Users\diogo\miniconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 63, 63, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,306,052 (12.61 MB)

 Trainable params: 3,305,604 (12.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=2)
]

steps_per_epoch = train_gen_custom.n // train_gen_custom.batch_size

history = optimized_cnn.fit(
    train_gen_custom,
    validation_data=(X_val, y_val),
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

C:\Users\diogo\miniconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
945/945 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - accuracy: 0.7442 - loss: 0.8796

MemoryError: Unable to allocate 4.75 GiB for an array with shape (25927, 128, 128, 3) and data type float32

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Predict
y_pred = optimized_cnn.predict(X_val)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_val, axis=1)

# Report
print(classification_report(y_true_labels, y_pred_labels, target_names=le.classes_))

# Confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
optimized_cnn.save('optimized_cnn.h5')

In [ ]:
def build_vgg_model(input_shape=(128, 128, 3), num_classes=4):
    base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base.layers:
        layer.trainable = False  # Freeze base layers

    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base.input, outputs=output)
    model.compile(optimizer=Adam(0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

vgg_model = build_vgg_model()
vgg_model.summary()


In [ ]:
def build_resnet_model(input_shape=(128, 128, 3), num_classes=4):
    base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base.layers:
        layer.trainable = False

    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base.input, outputs=output)
    model.compile(optimizer=Adam(0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

resnet_model = build_resnet_model()
resnet_model.summary()


In [ ]:
# Generator for training (from previous setup)
train_gen = augmentor.flow(X_train, y_train, batch_size=64, shuffle=True)

# Callbacks
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=2)
]

# Train VGG
history_vgg = vgg_model.fit(
    train_gen_vgg,
    steps_per_epoch=len(X_train) // 64,
    validation_data=(X_val, y_val),
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

# Train ResNet
history_resnet = resnet_model.fit(
    train_gen_resnet,
    steps_per_epoch=len(X_train) // 64,
    validation_data=(X_val, y_val),
    epochs=10,
    callbacks=callbacks,
    verbose=1
)


In [ ]:
plt.figure(figsize=(12, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['val_accuracy'], label='Custom CNN')
plt.plot(history_vgg.history['val_accuracy'], label='VGG16')
plt.plot(history_resnet.history['val_accuracy'], label='ResNet50')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['val_loss'], label='Custom CNN')
plt.plot(history_vgg.history['val_loss'], label='VGG16')
plt.plot(history_resnet.history['val_loss'], label='ResNet50')
plt.title('Validation Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:

resnet_model.save('resnet_model.h5')
vgg_model.save('vgg_model.h5')
